In [25]:
import yfinance as yf
import numpy as np
import plotly.express as px

In [26]:
df = yf.download("TCS.NS", period="6mo", multi_level_index=False)

[*********************100%***********************]  1 of 1 completed


In [27]:
df["SMA_5"] = df["Close"].rolling(window=5).mean()
df["SMA_20"] = df["Close"].rolling(window=20).mean()

df['Signal'] = 0
# df['Signal'][df['SMA_5'] > df['SMA_20']] = 1
df.loc[df['SMA_5'] > df['SMA_20'], "Signal"] = 1 
# df['Signal'][df['SMA_5'] < df['SMA_20']] = -1

df.loc[df['SMA_5'] < df['SMA_20'], "Signal"] = -1 
df['Position'] = df['Signal'].diff()
df.iloc[30:60]
df.loc[df["SMA_5"] > df["SMA_20"]]

,Close,High,Low,Open,Volume,SMA_5,SMA_20,Signal,Position
Date,,,,,,,,,
2024-12-16,4345.922363,4404.783864,4328.007743,4389.034916,1646224,4371.130469,4251.829675,1,1.0
2024-12-17,4260.582520,4363.393811,4250.247273,4340.803722,3225027,4350.646973,4267.037244,1,0.0
2024-12-18,4279.628906,4293.556785,4222.292895,4244.046224,1575404,4334.976758,4282.210364,1,0.0
2024-12-19,4204.870605,4295.525584,4195.126040,4214.812225,3015763,4298.941113,4292.006689,1,0.0
2025-01-14,4166.629883,4246.211378,4152.849554,4234.498195,3441709,4121.706348,4109.114587,1,2.0
2025-01-15,4182.920898,4189.663513,4141.432358,4151.816628,1653474,4149.503320,4105.231506,1,0.0
2025-01-16,4140.299805,4226.623918,4126.076824,4193.157404,2637705,4182.467773,4098.265051,1,0.0
2025-01-17,4124.299805,4160.600098,4100.049805,4159.899902,1761466,4167.583984,4094.236511,1,0.0
2025-01-20,4077.800049,4145.799805,4068.100098,4145.799805,1287799,4138.390088,4092.883276,1,0.0


In [ ]:
fig = px.line(
    df,
    x=df.index,
    y=["Close", "SMA_5", "SMA_20"],
    labels={"value": "Price", "variable": "Legend"},
    title="TCS - SMA Crossover Strategy",
)

fig.add_scatter(
    x=df[df["Position"] == 2].index,
    y=df.loc[df["Position"] == 2, "SMA_5"],
    mode="markers",
    marker=dict(symbol="triangle-up", size=20, color="green"),
    name="Buy Signal",
)

fig.add_scatter(
    x=df[df["Position"] == -2].index,
    y=df.loc[df["Position"] == -2, "SMA_20"],
    mode="markers",
    marker=dict(symbol="triangle-down", size=20, color="red"),
    name="Sell Signal",
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Price",
    legend_title="Legend",
    template="plotly_white",
    height=600,
    width=1000,
)
fig